# ETL
Load necessary libraries and movies metadata csv

In [30]:
import pandas as pd
import json

# Load the dataset
file_path = 'data/tmdb_5000_movies.csv'
df = pd.read_csv(file_path)

Drop non-relevant columns and drop columns being parsed out into separate new dataframes

In [31]:
# Drop the specified columns
df = df.drop(columns=['homepage', 'original_title', 'production_countries', 'spoken_languages'])

# Drop the fields that have been parsed out to new DataFrames (genres, keywords, production_companies)
movie_metadata_df = df.drop(columns=['genres', 'keywords', 'production_companies'])

Format currency related columns into dollar format, vote count to thousands format, popularity score to single decimal point, and filter dataset where status = 'Released'

In [32]:
# Format the 'budget' and 'revenue' fields as currency using string formatting
movie_metadata_df['budget'] = movie_metadata_df['budget'].apply(lambda x: f"${x:,.0f}" if pd.notnull(x) else x)
movie_metadata_df['revenue'] = movie_metadata_df['revenue'].apply(lambda x: f"${x:,.0f}" if pd.notnull(x) else x)

# Format the 'vote_count' field to thousands (K) format
movie_metadata_df['vote_count'] = movie_metadata_df['vote_count'].apply(lambda x: f"{x/1000:.1f}K" if pd.notnull(x) else x)

# Format the 'popularity' field to one decimal point
movie_metadata_df['popularity'] = movie_metadata_df['popularity'].apply(lambda x: f"{x:.1f}" if pd.notnull(x) else x)

# Filter the DataFrame where 'status' is 'Released' and keep the name 'movie_metadata_df'
movie_metadata_df = movie_metadata_df[movie_metadata_df['status'] == 'Released']

Parse out ID and Name from genre, keywords, production company fields in dataset into column format and create separate dataframes. Keep movie_ID as primary key to join to other dataframes. Drop any records with missing data.

In [33]:
# Function to parse the 'genres' column and create separate rows for each genre
def parse_genres(row):
    try:
        # Parse the JSON string from the 'genres' column
        genres = json.loads(row['genres'])
        # Create a list of tuples (movie_id, genre_id, genre_name)
        return [(row['id'], genre['id'], genre['name']) for genre in genres]
    except json.JSONDecodeError:
        return []

# Function to parse the 'keywords' column and create separate rows for each keyword
def parse_keywords(row):
    try:
        # Parse the JSON string from the 'keywords' column
        keywords = json.loads(row['keywords'])
        # Create a list of tuples (movie_id, keyword_id, keyword_name)
        return [(row['id'], keyword['id'], keyword['name']) for keyword in keywords]
    except json.JSONDecodeError:
        return []

# Function to parse the 'production_companies' column and create separate rows for each company
def parse_production_companies(row):
    try:
        # Parse the JSON string from the 'production_companies' column
        companies = json.loads(row['production_companies'])
        # Create a list of tuples (movie_id, company_id, company_name)
        return [(row['id'], company['id'], company['name']) for company in companies]
    except json.JSONDecodeError:
        return []

# Apply the parsing functions to the 'genres', 'keywords', and 'production_companies' columns
genres_list = df.apply(parse_genres, axis=1).explode()
keywords_list = df.apply(parse_keywords, axis=1).explode()
companies_list = df.apply(parse_production_companies, axis=1).explode()

# Create new DataFrames from the parsed lists
genre_df = pd.DataFrame(genres_list.tolist(), columns=['movie_id', 'genre_id', 'genre_name'])
keyword_df = pd.DataFrame(keywords_list.tolist(), columns=['movie_id', 'keyword_id', 'keyword_name'])
production_company_df = pd.DataFrame(companies_list.tolist(), columns=['movie_id', 'company_id', 'company_name'])

# Drop any rows with missing data (in case of malformed or missing information)
genre_df.dropna(inplace=True)
keyword_df.dropna(inplace=True)
production_company_df.dropna(inplace=True)

# Display the first few rows of the new DataFrames
print("Genres DataFrame:")
print(genre_df.head())

print("\nKeywords DataFrame:")
print(keyword_df.head())

print("\nProduction Companies DataFrame:")
print(production_company_df.head())

print("Movie Metadata DataFrame:")
print(movie_metadata_df.head())

Genres DataFrame:
   movie_id  genre_id       genre_name
0   19995.0      28.0           Action
1   19995.0      12.0        Adventure
2   19995.0      14.0          Fantasy
3   19995.0     878.0  Science Fiction
4     285.0      12.0        Adventure

Keywords DataFrame:
   movie_id  keyword_id   keyword_name
0   19995.0      1463.0  culture clash
1   19995.0      2964.0         future
2   19995.0      3386.0      space war
3   19995.0      3388.0   space colony
4   19995.0      3679.0        society

Production Companies DataFrame:
   movie_id  company_id                            company_name
0   19995.0       289.0                 Ingenious Film Partners
1   19995.0       306.0  Twentieth Century Fox Film Corporation
2   19995.0       444.0                      Dune Entertainment
3   19995.0       574.0                Lightstorm Entertainment
4     285.0         2.0                    Walt Disney Pictures
Movie Metadata DataFrame:
         budget      id original_language  \
0  $2

Load Oscar award cdv dataset

In [35]:
# Define the file path based on your working directory
file_path = 'data/oscars_df.csv'

# Load the CSV file into a new DataFrame
oscars_df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to confirm it's loaded correctly
print(oscars_df.head())


   Unnamed: 0                 Film Oscar Year Film Studio/Producer(s)  \
0           0                Wings    1927/28    Famous Players-Lasky   
1           1           7th Heaven    1927/28                     Fox   
2           2           The Racket    1927/28       The Caddo Company   
3           3  The Broadway Melody    1928/29     Metro-Goldwyn-Mayer   
4           4                Alibi    1928/29     Feature Productions   

     Award  Year of Release  Movie Time            Movie Genre  IMDB Rating  \
0   Winner             1927         144      Drama,Romance,War          7.5   
1  Nominee             1927         110          Drama,Romance          7.7   
2  Nominee             1928          84  Crime,Drama,Film-Noir          6.7   
3   Winner             1929         100  Drama,Musical,Romance          5.7   
4  Nominee             1929          91   Action,Crime,Romance          5.8   

  IMDB Votes  ... Tomatometer Status Tomatometer Rating Tomatometer Count  \
0     12,

Merge the oscar dataset with the movies metadata dataframe. Add a new column in the movie metadata dataframe called Award.

In [41]:
# Perform the merge/join operation, matching 'Film' from oscar_df with 'title' from movie_metadata_df
# Only the 'award' column from oscar_df will be added to movie_metadata_df
movie_metadata_df = movie_metadata_df.merge(oscars_df[['Film', 'Award']], 
                                            how='left', 
                                            left_on='title', 
                                            right_on='Film')

# Drop the 'Film' column from the merge (since it's the same as 'title')
movie_metadata_df = movie_metadata_df.drop(columns=['Film'])

# Ensure 'award' is the last column
award_column = movie_metadata_df.pop('Award')
movie_metadata_df['Award'] = award_column

# Display the first few rows of the updated DataFrame
print(movie_metadata_df.head())

         budget      id original_language  \
0  $237,000,000   19995                en   
1  $300,000,000     285                en   
2  $245,000,000  206647                en   
3  $250,000,000   49026                en   
4  $260,000,000   49529                en   

                                            overview popularity release_date  \
0  In the 22nd century, a paraplegic Marine is di...      150.4   2009-12-10   
1  Captain Barbossa, long believed to be dead, ha...      139.1   2007-05-19   
2  A cryptic message from Bond’s past sends him o...      107.4   2015-10-26   
3  Following the death of District Attorney Harve...      112.3   2012-07-16   
4  John Carter is a war-weary, former military ca...       43.9   2012-03-07   

          revenue  runtime    status  \
0  $2,787,965,087    162.0  Released   
1    $961,000,000    169.0  Released   
2    $880,674,609    148.0  Released   
3  $1,084,939,099    165.0  Released   
4    $284,139,100    132.0  Released   

       

Load the credits parquet file. Parse out the cast field with only the first ID and name in the string and create new columns along with the movie_id and title columns. Create a new dataframe. 

In [44]:
# Load the parquet file
file_path = 'data/credits_5000.parquet'
credits_df = pd.read_parquet(file_path)

# Function to parse the 'cast' column and extract only the first 'id' and 'name'
def parse_first_cast(row):
    try:
        # Parse the JSON string from the 'cast' column
        cast_list = json.loads(row['cast'])
        if cast_list:
            # Return the first cast member's id and name
            return (row['movie_id'], row['title'], cast_list[0]['id'], cast_list[0]['name'])
        else:
            return (row['movie_id'], row['title'], None, None)
    except (json.JSONDecodeError, IndexError):
        return (row['movie_id'], row['title'], None, None)

# Apply the parsing function to extract only the first cast member
first_cast_list = credits_df.apply(parse_first_cast, axis=1)

# Create a new DataFrame from the parsed cast list and rename it to 'credits_df'
credits_df = pd.DataFrame(first_cast_list.tolist(), columns=['movie_id', 'title', 'cast_id', 'cast_name'])

# Drop any rows with missing data (if any)
credits_df.dropna(inplace=True)

# Display the first few rows of the new DataFrame
print(credits_df.head())

   movie_id                                     title  cast_id  \
0     19995                                    Avatar  65731.0   
1       285  Pirates of the Caribbean: At World's End     85.0   
2    206647                                   Spectre   8784.0   
3     49026                     The Dark Knight Rises   3894.0   
4     49529                               John Carter  60900.0   

         cast_name  
0  Sam Worthington  
1      Johnny Depp  
2     Daniel Craig  
3   Christian Bale  
4    Taylor Kitsch  
